In [1]:
from gensim import models
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader
import sys,os
sys.path.append( os.path.join(".."))
from UtilityFunctions import CommonHelpers,PreprocessHelpers,FeatureEngineering
import tsne
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
import RunModels

/home/sharath/anaconda3/envs/fall21/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sharath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sharath/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sharath/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sharath/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
RAW_BOOK_FOLDER = os.path.join("..","data_collection","raw_books")
PREPROCESSED_STORE_LOC = os.path.join("..","data","preprocessed")

In [7]:
google_corpus_path  = "../models/GoogleNews-vectors-negative300.bin"
model_google = KeyedVectors.load_word2vec_format(google_corpus_path,binary=True)

In [4]:
list_of_words = CommonHelpers.load_pickle("../data/preprocessed/hundred_books_preprocessed.pickle")
book_names =  CommonHelpers.load_pickle("../data/preprocessed/hundred_names.pickle")


In [5]:
train_vectors = FeatureEngineering.get_avg_feature_vectors(list_of_words,model_google,300)

Fitting a random forest to labeled training data...


In [11]:
def get_number_id(file_name):
    import re 
    regex = r"([a-z_A-Z]+)(\d+)"
    match = re.search(regex, file_name)
    if match!=None:
        return match.group(2)

In [3]:
df = pd.read_csv("../data/pg_catalog.csv")
df["category"] = df["LoCC"].str[:1]

/home/sharath/anaconda3/envs/fall21/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,Text#,Type,Issued,Title,Language,Authors,Subjects,LoCC,Bookshelves,category
0,1,Text,1971-12-01,The Declaration of Independence of the United ...,en,"Jefferson, Thomas, 1743-1826","United States -- History -- Revolution, 1775-1...",E201; JK,Politics; American Revolutionary War; United S...,E
1,2,Text,1972-12-01,The United States Bill of Rights\r\nThe Ten Or...,en,United States,Civil rights -- United States -- Sources; Unit...,JK; KF,Politics; American Revolutionary War; United S...,J
2,3,Text,1973-11-01,John F. Kennedy's Inaugural Address,en,"Kennedy, John F. (John Fitzgerald), 1917-1963",United States -- Foreign relations -- 1961-196...,E838,NaN,E
3,4,Text,1973-11-01,Lincoln's Gettysburg Address\r\nGiven November...,en,"Lincoln, Abraham, 1809-1865",Consecration of cemeteries -- Pennsylvania -- ...,E456,US Civil War,E
4,5,Text,1975-12-01,The United States Constitution,en,United States,United States -- Politics and government -- 17...,JK; KF,United States; Politics; American Revolutionar...,J


In [6]:
categories = df[df["Text#"].isin([1,3,4])].iloc[:,9].tolist()
categories

['E', 'E', 'E']

In [13]:
category_list = [int(get_number_id(each)) for each in book_names]

In [19]:
categories = df.iloc[category_list,9].tolist()
titles = df.iloc[category_list,3].tolist()

CommonHelpers.dump_pickle("temp_vectors.pickle",train_vectors)
CommonHelpers.dump_pickle("temp_labels.pickle",categories)

In [45]:
doc_vec_train={}
genres_train={}
for i in range(1,len(titles)):
    doc_vec_train[titles[i]]= train_vectors[i]
    genres_train[titles[i]] = categories[i]

In [48]:
tsne.plot_vectors(model_google,doc_vec_train,genres_train)

AttributeError: module 'tsne' has no attribute 'plot_vectors'

In [17]:
forest = RandomForestClassifier(n_estimators = 100)

print("Fitting a random forest to labeled training data...")
forest = RunModels.Run_Model(forest,train_vectors,categories)
forest.run_model()
result = forest.get_predictions()
result

Fitting a random forest to labeled training data...


array(['P', 'E', 'T', 'T', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P',
       'T', 'P', 'P', 'P', 'P', 'P', 'P'], dtype='<U1')

In [18]:
print(forest.getMetrics())

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].